In [2]:
pip install openml

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.4/160.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.1/95.1 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 56.3 MB/s eta 0:00:00
  Created wheel for liac-arff: filename=liac_arff-2.5.0-py3-none-any.whl size=11716 sha256=f69352717ae368558cf30534001dc378a52246586b5531b16e7c07722e58c633
  Stored in directory: /root/.cache/pip/wheels/00/23/31/5e562fce1f95aabe57f2a7320d07433ba1cd152bcde2f6a002
Successfully built liac-arff


In [3]:
pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 4.0 MB/s eta 0:00:00


In [ ]:
from openml.datasets import get_dataset

dataset = get_dataset(42706)
X, y, _, _ = dataset.get_data(target=dataset.default_target_attribute)

print(X.dtypes.value_counts())

uint8    197631
int64      2369
Name: count, dtype: int64


In [17]:
import openml
import pandas as pd
# import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

class OpenMLPipeline:
    def __init__(self, dataset_id, test_size=0.2, random_state=42):
        self.dataset_id = dataset_id
        self.test_size = test_size
        self.random_state = random_state
        self.dataset = None
        self.features = None
        self.labels = None
        self.preprocessor = None

    def fetch_data(self):
        print(f"Fetching dataset {self.dataset_id} from OpenML...")
        dataset = openml.datasets.get_dataset(self.dataset_id)
        X, y, _, _ = dataset.get_data(target=dataset.default_target_attribute)
        self.features = X
        self.labels = y
        print("fetch data:", X.shape)
        return X, y

    def preprocess_data(self):
        if self.features is None or self.labels is None:
            raise ValueError("Dataset not loaded. Call fetch_data() first.")

        numeric_features = self.features.select_dtypes(include=['int64', 'float64', 'uint8']).columns
        categorical_features = self.features.select_dtypes(include=['object', 'category']).columns

        numeric_transformer = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler())
        ])

        categorical_transformer = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OneHotEncoder(handle_unknown='ignore'))
        ])

        self.preprocessor = ColumnTransformer(
            transformers=[
                ('num', numeric_transformer, numeric_features),
                ('cat', categorical_transformer, categorical_features)
            ]
        )

        X_processed = self.preprocessor.fit_transform(self.features)
        print("Feature Data Types:\n", self.features.dtypes)
        print("Original features:", list(self.features.columns))
        print("Numeric features used:", list(numeric_features))
        print("Categorical features used:", list(categorical_features))
        return X_processed, self.labels

    def split_data(self, X, y):
        return train_test_split(X, y, test_size=self.test_size, random_state=self.random_state)

    def save_to_csv(self, X, y, X_train, X_test, y_train, y_test):
        df_full = pd.DataFrame(X)
        df_full['target'] = y.values
        df_full.to_csv("processed_dataset.csv", index=False)

        df_train = pd.DataFrame(X_train)
        df_train['target'] = y_train.values
        df_train.to_csv("train_dataset.csv", index=False)

        df_test = pd.DataFrame(X_test)
        df_test['target'] = y_test.values
        df_test.to_csv("test_dataset.csv", index=False)

        print("Processed datasets saved: processed_dataset.csv, train_dataset.csv, test_dataset.csv")

    def run_pipeline(self):
        self.fetch_data()
        X_processed, y = self.preprocess_data()
        X_train, X_test, y_train, y_test = self.split_data(X_processed, y)
        self.save_to_csv(X_processed, y, X_train, X_test, y_train, y_test)
        print("Data pipeline completed.")
        return X_train, X_test, y_train, y_test

if __name__ == "__main__":
    pipeline = OpenMLPipeline(dataset_id=50)  # Example dataset ID
    X_train, X_test, y_train, y_test = pipeline.run_pipeline()
    print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")


Fetching dataset 50 from OpenML...
fetch data: (958, 9)
Feature Data Types:
 top-left-square         category
top-middle-square       category
top-right-square        category
middle-left-square      category
middle-middle-square    category
middle-right-square     category
bottom-left-square      category
bottom-middle-square    category
bottom-right-square     category
dtype: object
Original features: ['top-left-square', 'top-middle-square', 'top-right-square', 'middle-left-square', 'middle-middle-square', 'middle-right-square', 'bottom-left-square', 'bottom-middle-square', 'bottom-right-square']
Numeric features used: []
Categorical features used: ['top-left-square', 'top-middle-square', 'top-right-square', 'middle-left-square', 'middle-middle-square', 'middle-right-square', 'bottom-left-square', 'bottom-middle-square', 'bottom-right-square']
Processed datasets saved: processed_dataset.csv, train_dataset.csv, test_dataset.csv
Data pipeline completed.
Train shape: (766, 27), Test sha

In [ ]:
import openml
import pandas as pd
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

class OpenMLPipeline:
    def __init__(self, dataset_id, test_size=0.2, random_state=42):
        self.dataset_id = dataset_id
        self.test_size = test_size
        self.random_state = random_state
        self.dataset = None
        self.features = None
        self.labels = None
        self.preprocessor = None

    def fetch_data(self):
        print(f"Fetching dataset {self.dataset_id} from OpenML...")
        dataset = openml.datasets.get_dataset(self.dataset_id)
        X, y, _, _ = dataset.get_data(target=dataset.default_target_attribute)
        self.features = X
        self.labels = y
        print("fetch data:", X.shape)
        return X, y

    def preprocess_data(self):
      if self.features is None or self.labels is None:
        raise ValueError("Dataset not loaded. Call fetch_data() first.")

      # Identify numeric and categorical features
      numeric_features = self.features.select_dtypes(include=['int64', 'float64', 'uint8']).columns
      categorical_features = self.features.select_dtypes(include=['object', 'category']).columns

      # Apply Target Encoding separately
      if not categorical_features.empty:
        target_encoder = ce.TargetEncoder(cols=categorical_features)
        self.features[categorical_features] = target_encoder.fit_transform(self.features[categorical_features], self.labels)

      # Define numeric transformer pipeline
      numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
      ])

      # Create ColumnTransformer for numeric features only (categoricals are already encoded)
      self.preprocessor = ColumnTransformer(
        transformers=[
          ('num', numeric_transformer, numeric_features)
        ]
      )

      # Transform numeric features
      X_processed = self.preprocessor.fit_transform(self.features)

      print("Feature Data Types:\n", self.features.dtypes)
      print("Numeric features used:", list(numeric_features))
      print("Categorical features used (Target Encoded):", list(categorical_features))
      print("Processed dataset shape:", X_processed.shape)

      return X_processed, self.labels

    def split_data(self, X, y):
        return train_test_split(X, y, test_size=self.test_size, random_state=self.random_state)

    def save_to_csv(self, X, y, filename="processed_dataset.csv"):
        df = pd.DataFrame(X)
        df['target'] = y.values
        df.to_csv(filename, index=False)
        print(f"Processed dataset saved to {filename}")

    def run_pipeline(self):
        self.fetch_data()
        X_processed, y = self.preprocess_data()
        self.save_to_csv(X_processed, y)
        X_train, X_test, y_train, y_test = self.split_data(X_processed, y)
        print("Data pipeline completed.")
        return X_train, X_test, y_train, y_test

if __name__ == "__main__":
    pipeline = OpenMLPipeline(dataset_id=31)
    X_train, X_test, y_train, y_test = pipeline.run_pipeline()
    print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")


Fetching dataset 31 from OpenML...
fetch data: (1000, 20)


AttributeError: 'numpy.ndarray' object has no attribute 'groupby'